# Lab 11 – AVL & Red-Black Trees

**Goals:**
- Understand *self-balancing* BST ideas: AVL rotations and RBT invariants.
- Implement an **AVL tree** with insert + rotations; verify height-balance.
- Practice **reasoning** about Red-Black Tree properties with a small simulator stub.


## Definitions

- **AVL tree:** a BST where for every node, `|height(left) - height(right)| ≤ 1`. Rebalance by rotations after insert/delete.
- **Rotations:** single and double rotations restore balance.
- **Red-Black Tree (RBT):** BST with node colors (red/black) and invariants ensuring O(log n) height.


## Binary Search Trees

A **Binary Search Tree** (BST) is a binary tree with an additional ordering property:

- Every node has at most two children (left and right).
- For any node with key k:
    + All keys in the left subtree are < k
    + All keys in the right subtree are > k
- There are no duplicate keys (for this course).

BSTs allow efficient searching, insertion, and deletion: O(h) time, where h is the tree height.

Balanced trees make h = O(log n), but ordinary BSTs can degrade to h = O(n) if they become skewed.
(We will fix that with AVL / RB trees.)

1. Consider inserting the following keys into an empty BST in the given order:

`[30, 15, 40, 10, 20, 35, 50]`

a) Draw the BST after each insertion (on paper).
b) What is the height of the final tree?

2. Consider inserting the following keys into an empty BST:

`[10, 9, 8, 7, 6, 5, 4]`

a) Draw the BST after each insertion (on paper).
b) What is the height of the final tree?

3. Implement the following methods for basic BST Class.

- insert
- search - returns `True` if `key` was found

In [ ]:
class BasicBST:
    def __init__(self, root_key):
        self.key = root_key
        self.left_child = None
        self.right_child = None

    def get_left_child(self):
        return self.left_child

    def get_right_child(self):
        return self.right_child

    def set_root_val(self, obj):
        self.key = obj

    def get_root_val(self):
        return self.key

    # TODO
    def insert(self, key):
        assert NotImplementedError

    # TODO
    def search(self, key) -> bool:
        assert NotImplementedError

    # TODO
    def delete(self, key) -> bool:
        assert NotImplementedError

    def __str__(self):
        """Return string representation of tree in list form."""
        left = str(self.left_child) if self.left_child else "[]"
        right = str(self.right_child) if self.right_child else "[]"
        return f"[{self.key}, {left}, {right}]"


nums = [30, 15, 40, 10, 20, 35, 50]
bst = BasicBST(nums.pop(0))
for n in nums:
    bst.insert(n)

print(bst)

4. Build two BSTs, one for each of the lists in problems (1) and (2) above.

5. Write a BST sort function `BST_sort()` that takes a list, builds a BST, then uses an appropriate traversal method to output a sorted list.

6. Use your work on 1-5 to outline complexity for the following BST operations.

| Operation | Best Case | Worst Case |
|---|---|---|
| `insert`   | ? |  ? |
| `search`   | ? |  ? |
| `delete`   | ? |  ? |
| `BST_sort` | ? | ? |




The above illustrates the central problem:

- BSTs only perform well if kept balanced.
- But ordinary insertions do not guarantee balance.

Therefore, we introduce self-balancing BSTs, like:

- AVL Trees (strict height balancing via rotations)
- Red-Black Trees (color rules guarantee approximate balance)

Both aim to keep height ≈ O(log n) for efficient search, insert, and delete.

## AVL Trees (optional)

An AVL Tree is a self-balancing Binary Search Tree. It keeps the tree height as close to O(log n) as possible by performing rotations whenever insertions or deletions create imbalance.

AVL trees were invented by Adelson-Velsky and Landis (1962)—hence AVL.

For every node in the tree:

> balance(node) = height(left subtree) - height(right subtree)

A node is balanced if:

> balance(node) is -1, 0, or 1

Balance is mainted by rotations, small, loacl tree surgery that reduces height of tall subtree but keeps inorder traversal the same (BST property). A rotation moves a deep node closer to the root and a shallow node down, reducing height imbalance.

In this way, AVL Trees maintain the following **structural invariants**:

- Rotations must preserve BST ordering.
- Rotations must update heights correctly.
- The root pointer must update after rotations affecting the root.

There are exactly four rotation cases. They depend on where the imbalance occurs (Left or Right) and where the inserted/deleted key goes (Left or Right).

1. Left-Left (LL) - New key inserted into left subtree of the left child.  
Fix: Right Rotation.

2. Right-Right (RR) - New key inserted into right subtree of the right child.
Fix: Left Rotation.

3. Left-Right (LR) - New key inserted into right subtree of the left child.
Fix: Left rotation on left child, then right rotation on node

4. Right-Left (RL) - New key inserted into left subtree of the right child.
Fix: Right rotation on right child, then left rotation on node

These four cases guarantee that the tree will again satisfy the AVL balance condition.

For tjos ta


**Task 1 – Height and Balance Factor:** Implement the following helper functions using the AVL height and balance rules:
- `_height`
- `_update_height`
- `_balance_factor`

**Task 2 – Rotations:** Implement the AVL rotation helpers:
- `_rotate_right`
- `_rotate_left`

Use the rotation diagrams and step-by-step rotation instructions provided above.

**Task 3 – Insertion with Rebalancing:** Implement `_insert` with the following steps:

1. Perform a **standard BST insert**.  
2. **Update the height** of each node on the way back up.  
3. **Compute the balance factor** at each node.  
4. If the node is unbalanced, detect which of the four cases applies:
   - Left-Left (LL)
   - Left-Right (LR)
   - Right-Right (RR)
   - Right-Left (RL)
5. Apply the **appropriate rotation(s)** to restore balance.

**Task 4 – Inorder Traversal:** Implement `_inorder_keys` and confirm that:

- `inorder_keys()` returns a list of keys  
- The list is sorted, demonstrating the BST property of your AVL tree.

In [ ]:
class AVLNode:
    __slots__ = ("key", "left", "right", "height")

    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        # We will use height = 1 for a leaf node
        self.height = 1


class AVL_Tree:
    """
    Basic AVL Tree implementation with:
    - AVLNode as the node type
    - self.root as the root pointer
    - insert(key) to add a new key
    - inorder_keys() to return sorted keys
    """

    def __init__(self):
        """Initialize an empty AVL tree."""
        self.root = None

    # -------------------------
    # Task 1: Height / Balance
    # -------------------------
    def _height(self, node):
        """
        Return the height of a node.
        Empty subtree should have height 0.
        Non-empty nodes use node.height.
        """
        raise NotImplementedError("Task 1: Implement _height")

    def _update_height(self, node):
        """
        Recompute node.height from the heights of its children.
        Formula: 1 + max(height(left), height(right))
        """
        raise NotImplementedError("Task 1: Implement _update_height")

    def _balance_factor(self, node):
        """
        Return the balance factor of a node:
        height(left subtree) - height(right subtree)
        """
        raise NotImplementedError("Task 1: Implement _balance_factor")

    # -------------------------
    # Task 2: Rotations
    # -------------------------
    def _rotate_right(self, y):
        """
        Perform a right rotation around node y.

                y                  x
               / \                / \
              x   T3    -->      T1  y
             / \                    / \
            T1  T2                 T2  T3

        Steps:
        - Let x = y.left
        - Move x.right (T2) to y.left
        - Make y the right child of x
        - Update heights of y, then x
        - Return new root (x)
        """
        raise NotImplementedError("Task 2: Implement _rotate_right")

    def _rotate_left(self, x):
        """
        Perform a left rotation around node x.

            x                     y
           / \                   / \
          T1  y      -->        x  T3
             / \               / \
            T2  T3            T1 T2

        Steps:
        - Let y = x.right
        - Move y.left (T2) to x.right
        - Make x the left child of y
        - Update heights of x, then y
        - Return new root (y)
        """
        raise NotImplementedError("Task 2: Implement _rotate_left")

    # ------------------------------------
    # Task 3: AVL Insert (recursive)
    # ------------------------------------
    def _insert(self, node, key):
        """
        Recursively insert key into the subtree rooted at node.
        1) Do a standard BST insertion.
        2) Update this node's height.
        3) Compute balance factor to detect imbalance.
        4) Fix with appropriate rotation(s):
            - Left-Left  (LL)
            - Left-Right (LR)
            - Right-Right (RR)
            - Right-Left (RL)
        Return the (possibly new) root of this subtree.
        """
        raise NotImplementedError("Task 3: Implement _insert")

    def insert(self, key):
        """
        Public insert method.
        Updates self.root by calling the recursive _insert.
        No duplicate keys for this lab.
        """
        self.root = self._insert(self.root, key)

    # ------------------------------------
    # Task 4: Inorder Traversal
    # ------------------------------------
    def _inorder_keys(self, node):
        """
        Return a list of keys from an inorder traversal
        of the subtree rooted at node.
        (Left, Node, Right)
        """
        raise NotImplementedError("Task 4: Implement _inorder_keys")

    def inorder_keys(self):
        """
        Return a sorted list of all keys in the AVL tree
        using inorder traversal of self.root.
        """
        return self._inorder_keys(self.root)

---

## Self‑Assessment
Please mark one option by editing the brackets to `[x]`:

- [ ] **10** – I completed all of this work on my own (learning from in‑class ideas/approaches).
- [ ] **8** – I completed most on my own, with some out‑of‑class help (peers/online).
- [ ] **6** – I needed significant help (peers/online/AI) to complete parts.
- [ ] **4** – I mostly copied code from others/AI and **do not** fully understand it.
- [ ] **2** – I copied almost everything without attempting to understand it.
